In [5]:
import create_dict
import torch
import os
import json

# Clean Data

In [ ]:
folder_name = '/data'
create_dict.clean_obj(folder_name)

# Generate Dataset

In [ ]:
input_folder = '/data'
suffix = '_test' #suffix will append data_trainxxxx.json
max_elements_per_class = 5
create_dict.create_dictionary(input_folder, max_elements_per_class, suffix)

# Initialize Network

In [2]:
import SiameseNet

batch_size = 2
net = SiameseNet.SiameseNet(batch_size)
suffix = '_test' # comment in if not coming from generating the dataset


path_to_params = "models/test.pt" # if file does not exist or is empty it starts from untrained and later saves to the file

working_dir = os.getcwd()
data_dir_train = os.path.join(working_dir, 'data_train'+suffix+'.json')
data_dir_val = os.path.join(working_dir, 'data_val'+suffix+'.json')
class_dir = os.path.join(working_dir, 'class_dict'+suffix+'.json')

if os.path.isfile(path_to_params):
    if os.stat(path_to_params).st_size != 0:
        net.load_state_dict(torch.load(path_to_params))  #Loads pretrained net if file exists and if not empty
else:
    open(path_to_params, "x") #Creates parameter file if it does not exist

# Train Network

In [ ]:
#training parameters

writer_suffix = 'testing'
margin = 0.5
num_epochs = 1
print_batch = 1
lr = 0.0001



In [ ]:
net = SiameseNet.train(net, num_epochs, margin, lr, print_batch, 
                       data_dir_train, data_dir_val, writer_suffix, path_to_params, working_dir)

# Validation

In [3]:
writer_suffix = 'Val'
margin = 0.5

y_true, y_pred, ids, shape, description = SiameseNet.retrieval(net, data_dir_val, working_dir)

8 val triplets generated
Loaded 5032 word vectors.
Number of validation triplets: 4
0 [2 5 3 7 1]
1 [2 5 3 7 1]
2 [2 5 3 7 1]
3 [2 5 3 4 1]
4 [2 5 3 7 1]
5 [2 5 3 7 1]
6 [2 5 3 7 1]
7 [2 5 3 7 1]


In [6]:
with open(data_dir_val, 'r') as fp:
    data_val = json.load(fp)

max_show = 3
for i, key in enumerate(ids):
    if (i>=max_show):
        break
    print("Sample: ID:", key, " Description:", data_val[key][1])
    print("1N: ID", ids[y_pred[i][0]]) #todo get picture
    print("2N: ID", ids[y_pred[i][1]]) #todo get picture

Sample: ID: cb38405e384c79c05e4e385b035691bb  Description: ['this', 'is', 'a', 'brown', 'tub']
1N: ID 2388c99892c2185268d1b9a1d97e2846
2N: ID e5b583b0cf478984d781754d41e2576e
Sample: ID: 555f9430c2ca273ccb2a965e75be701c  Description: ['this', 'is', 'a', 'brown', 'and', 'red', 'table']
1N: ID 2388c99892c2185268d1b9a1d97e2846
2N: ID e5b583b0cf478984d781754d41e2576e
Sample: ID: 2388c99892c2185268d1b9a1d97e2846  Description: ['this', 'is', 'up', 'lamp', 'around', 'a', 'blue', 'pole', 'with', 'no', 'lamp', 'shade']
1N: ID 2388c99892c2185268d1b9a1d97e2846
2N: ID e5b583b0cf478984d781754d41e2576e
